In [ ]:
from huggingface_hub import login

# Provide your Hugging Face token
login(token="your token here")


C:\Users\pooji\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Model Loading:

The script is loading the Llama-3.2-3B-Instruct model using Hugging Face's transformers library.

* AutoTokenizer is used to load a pre-trained tokenizer
* AutoModelForCausalLM loads the pre-trained language model for text generation.

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model     = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

Loading checkpoint shards: 100%|██████████| 2/2 [00:53<00:00, 26.51s/it]


### Importing all the required libraries

### Load Poems

In [3]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
import torch
import collections
from collections import Counter
import math
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from nltk.util import ngrams
import pronouncing
from nltk.corpus import cmudict
from collections import defaultdict
import re


### Dataset Handling:

The dataset is loaded from a CSV file containing prompts and generated poems. It includes all the poems generated by six models: Llama 3.2 3B Instruct Q8, Mistral 7B Instruct Q4, Gemma 2 2B Q4, QWEN2 7B Instruct Q4, Openchat-3.5 7B, and Code Ninja 7B Q4. 

Below are the prompts used to generate three poems:
* Prompt 1: Write a Shakespearean sonnet about courage set on a battlefield with a determined tone. Use vivid imagery to convey strength and resilience.
* Prompt 2: Write a Shakespearean sonnet about wonder set in outer space with an awe-filled tone. Use vivid imagery to convey mystery and discovery.
* Prompt 3: Write a Shakespearean sonnet about loss set in an empty home with a somber tone. Use vivid imagery to convey grief and reflection.

In [4]:
file_path = "Model_gen_Poems.csv"
data = pd.read_csv(file_path)
print("Dataset Preview:")
print(data.head())
print("\nColumn Names:")
print(data.columns)
print("\nDataset Info:")
data.info()

Dataset Preview:
                                        Prompt/Model  \
0  Write a Shakespearean sonnet \nabout courage s...   
1  Write a Shakespearean sonnet \nabout wonder se...   
2  Write a Shakespearean sonnet \nabout loss set ...   

                            Llama 3.2 3B Instruct Q8  \
0  Fair battlefield, where valorous hearts do lie...   
1  Fairest cosmos, thou dost stretch thy might,\n...   
2  In vacant halls, where echoes whisper low,\nA ...   

                              Mistral 7B INSTRUCT Q4  \
0  Upon the field of battle, where chaos reigns,\...   
1  In the vast expanse of the cosmos, where stars...   
2  In the empty home, where laughter once was hea...   

                                       Gemma 2 2B Q4  \
0  Upon the field of strife, where blood doth sta...   
1  Upon the velvet canvas of the night,\nA millio...   
2  The dust motes dance in sunbeams, pale and thi...   

                                QWEN2 7B Instruct Q4  \
0  Amidst the clash of stee

In [5]:
print("\nColumn Names:")
print(data.columns)


Column Names:
Index(['Prompt/Model', 'Llama 3.2 3B Instruct Q8', 'Mistral 7B INSTRUCT Q4',
       'Gemma 2 2B Q4', 'QWEN2 7B Instruct Q4', 'Openchat-3.5 7B',
       'Code Ninja 7B Q4'],
      dtype='object')


In [6]:
print("\nDataset Info:")
data.info()


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Prompt/Model              3 non-null      object
 1   Llama 3.2 3B Instruct Q8  3 non-null      object
 2   Mistral 7B INSTRUCT Q4    3 non-null      object
 3   Gemma 2 2B Q4             3 non-null      object
 4   QWEN2 7B Instruct Q4      3 non-null      object
 5   Openchat-3.5 7B           3 non-null      object
 6   Code Ninja 7B Q4          3 non-null      object
dtypes: object(7)
memory usage: 300.0+ bytes


### Reference Poems

A list of reference poems is provided to evaluate the generated poem's quality.

The reference for the poem in prompt 1 is taken from the website AllPoetry[https://allpoetry.com/poems/about/Courage]. The poem is titled The Brave Ones and is authored by Guilty.

In [7]:
referenced_poem_prompt1 = (
    "The brave ones are the living"
    "grabbing life with both hands"
    "reckless adventures flying"
    "with not a care where they land"
    
    "The meek ones merely exist"
    "going unnoticed is their plan"
    "perhaps they will perish"
    "before they need to take a stand"
    
    "The brave ones believe"
    "in more than what they see"
    "the great unknown mystifies"
    "and life is yet to be revealed"
    
    "The meek ones pity their lot"
    "whatever they have is what they have got"
    "there is nothing grand"
    "in eating out of another’s hands"
    
    "Be bold, be brave and take a chance"
    "a glorious failure is better than a lofty dream"
    "never aspired for"
)

The reference for the poem in prompt 2 is taken from the website [https://poemsplease.com/12-poems-reflecting-the-wonder-of-space-exploration/]. The poem is titled Boundless Skies. 

In [8]:
referenced_poem_prompt2 = (
    "In the stillness of night, where silence reigns,"
    "A canvas spreads, inked with ethereal gains."
    "Stars like diamonds twinkle, shimmering light,"
    "Infinite stories held, ready for flight."
    "Comets trace paths, fiery trails that gleam,"
    "While galaxies swirl in the vastest dream."
    "Cosmic whispers call, beckoning the brave,"
    "To wander through wonders, the universe to save."
)

The reference for the poem in prompt 3 is taken from the website [https://statics.teams.cdn.office.net/evergreen-assets/safelinks/1/atp-safelinks.html]. The poem is titled The Empty House and is authored by Walter de La Mare.

In [9]:
referenced_poem_prompt3 = (
    "Whenever I walk to Suffern along the Erie track"
    "I go by a poor old farmhouse with its shingles broken and black."
    "I suppose I've passed it a hundred times, but I always stop for a minute"
    "And look at the house, the tragic house, the house with nobody in it."
    
    "I never have seen a haunted house, but I hear there are such things;"
    "That they hold the talk of spirits, their mirth and sorrowings."
    "I know this house isn't haunted, and I wish it were, I do;"
    "For it wouldn't be so lonely if it had a ghost or two."
    
    "This house on the road to Suffern needs a dozen panes of glass,"
    "And somebody ought to weed the walk and take a scythe to the grass."
    "It needs new paint and shingles, and the vines should be trimmed and tied;"
    "But what it needs the most of all is some people living inside."
    
    "If I had a lot of money and all my debts were paid"
    "I'd put a gang of men to work with brush and saw and spade."
    "I'd buy that place and fix it up the way it used to be"
    "And I'd find some people who wanted a home and give it to them free."
    
    "Now, a new house standing empty, with staring window and door,"
    "Looks idle, perhaps, and foolish, like a hat on its block in the store."
    "But there's nothing mournful about it; it cannot be sad and lone"
    "For the lack of something within it that it has never known."
    
    "But a house that has done what a house should do,"
    "a house that has sheltered life,"
    "That has put its loving wooden arms around a man and his wife,"
    "A house that has echoed a baby's laugh and held up his stumbling feet,"
    "Is the saddest sight, when it's left alone, that ever your eyes could meet."
    
    "So whenever I go to Suffern along the Erie track"
    "I never go by the empty house without stopping and looking back,"
    "Yet it hurts me to look at the crumbling roof and the shutters fallen apart,"
    "For I can't help thinking the poor old house is a house with a broken heart."
)


In [10]:
reference_poems = [referenced_poem_prompt1, referenced_poem_prompt2, referenced_poem_prompt3]

In [11]:
# Add a column for reference poems
data["Reference Poem"] = reference_poems

In [12]:
data.head()

,Prompt/Model,Llama 3.2 3B Instruct Q8,Mistral 7B INSTRUCT Q4,Gemma 2 2B Q4,QWEN2 7B Instruct Q4,Openchat-3.5 7B,Code Ninja 7B Q4,Reference Poem
0,Write a Shakespearean sonnet \nabout courage s...,"Fair battlefield, where valorous hearts do lie...","Upon the field of battle, where chaos reigns,\...","Upon the field of strife, where blood doth sta...","Amidst the clash of steel and blood, I stand u...","Upon the field of strife, where metal meets,\n...",Upon the battlefield where fierce armies clash...,The brave ones are the livinggrabbing life wit...
1,Write a Shakespearean sonnet \nabout wonder se...,"Fairest cosmos, thou dost stretch thy might,\n...","In the vast expanse of the cosmos, where stars...","Upon the velvet canvas of the night,\nA millio...","In the vast expanse of cosmic air, where stars...","Upon the canvas of the boundless sky,\nWhere d...","When first upon this boundless void I gazed,\n...","In the stillness of night, where silence reign..."
2,Write a Shakespearean sonnet \nabout loss set ...,"In vacant halls, where echoes whisper low,\nA ...","In the empty home, where laughter once was hea...","The dust motes dance in sunbeams, pale and thi...","In silent halls where shadows loom, \nThe echo...","Upon these hallowed walls, where echoes linger...","When shadows eclipse the sun's bright ray,\nAn...",Whenever I walk to Suffern along the Erie trac...


## Fluency

Evaluating fluency is important in poetry generation to ensure that the generated poems not only follow linguistic rules but also convey meaning smoothly. Several metrics can be used to assess the fluency of generated poetry:
*   BLEU (Bilingual Evaluation Understudy):
Definition  : BLEU is a metric commonly used in machine translation. It measures the precision of n-grams (contiguous sequences of n words) in the generated text against a reference text.
Use         : While it’s more commonly applied to translation tasks, it can be adapted to poetry evaluation by comparing n-grams between generated poems and reference poems.

*   METEOR (Metric for Evaluation of Translation with Explicit ORdering):
Definition  : METEOR is designed to address some of the shortcomings of BLEU, particularly by considering synonyms, stemming, and word order.
Use         : It evaluates the semantic and syntactic similarity between generated and reference text, which can be helpful for poetry generation.

*   Perplexity:
Definition  : Perplexity measures how well a model predicts the next word in a sequence. Lower perplexity indicates that the model is better at predicting text and is more "certain" in its predictions.
Use         : This can give an idea of how fluent or coherent the generated poem is, although it doesn't always align with creative quality.

*   Entropy:  
Definition  : Entropy quantifies the average uncertainty or information content in the text. It measures the diversity or randomness in token distributions, such as characters or words, in the generated output. Higher entropy indicates more diverse text, while lower entropy suggests repetitive or predictable output.
Use         : Entropy is valuable for evaluating poetry generation as it reflects the balance between creativity and structure. For poetry, moderate entropy often indicates well-crafted and varied text with coherent patterns, while extreme entropy values can signal overly repetitive or chaotic outputs. This metric helps compare models’ ability to produce diverse and engaging poems.  

In [13]:
# Evaluation Functions
def bleu_score(reference, candidate):
    return round(sentence_bleu([reference.split()], candidate.split()), 5)

def meteor_score_func(reference, candidate):
    # Tokenize both the reference and candidate
    reference_tokens = reference.split()  # Tokenize the reference poem
    candidate_tokens = candidate.split()  # Tokenize the generated poem
    return round(meteor_score([reference_tokens], candidate_tokens), 5)

def calculate_perplexity(text):
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        return round(torch.exp(loss).item(), 5)

def calculate_entropy(text):
    tokens = list(text)  # For word-level, use text.split()
    
    token_counts = Counter(tokens)
    total_tokens = len(tokens)
    
    probabilities = [count / total_tokens for count in token_counts.values()]
    
    entropy = -sum(p * math.log2(p) for p in probabilities)
    return round(entropy, 5)

In [14]:
# Specify the model column to evaluate
model_name_to_evaluate = "Llama 3.2 3B Instruct Q8"  # Replace with desired model column name

In [15]:
# Evaluate Scores
results = []
for index, row in data.iterrows():
    prompt = row["Prompt/Model"]
    reference_poem = row["Reference Poem"]
    generated_poem = row[model_name_to_evaluate]
    
    # Calculate scores
    bleu = bleu_score(reference_poem, generated_poem)
    meteor = meteor_score_func(reference_poem, generated_poem)
    perplexity = calculate_perplexity(generated_poem)
    entropy = calculate_entropy(generated_poem)
    
    # Append to results
    results.append({
        "Prompt": prompt,
        "BLEU": bleu,
        "METEOR": meteor,
        "Perplexity": perplexity,
        "Entropy": entropy  # Fixed key capitalization for clarity
    })

# Convert results to DataFrame for better visualization
results_df = pd.DataFrame(results)

# Calculate average entropy
average_entropy = results_df["Entropy"].mean()
average_perplexity = results_df["Perplexity"].mean()

# Add model name to the title
evaluation_title = f"Evaluation Results for Model: {model_name_to_evaluate}"

# Display Results
print(evaluation_title)
print(results_df)

# Display the average entropy
print(f"\nAverage Perplexity: {average_perplexity:.4f}")
print(f"\nAverage Entropy: {average_entropy:.4f}")


C:\Users\pooji\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\pooji\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\pooji\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-pa

Evaluation Results for Model: Llama 3.2 3B Instruct Q8
                                              Prompt  BLEU   METEOR  \
0  Write a Shakespearean sonnet \nabout courage s...   0.0  0.06501   
1  Write a Shakespearean sonnet \nabout wonder se...   0.0  0.12797   
2  Write a Shakespearean sonnet \nabout loss set ...   0.0  0.06231   

   Perplexity  Entropy  
0     3.53260  4.31690  
1     3.61225  4.29059  
2     3.25830  4.39931  

Average Perplexity: 3.4677

Average Entropy: 4.3356


BLEU: 
For all three prompts the BLEU score is 0.0, meaning the generated sonnets don't closely match the reference sonnets in terms of exact word sequences. This suggests that the model doesn’t replicate the exact phrases or wording from the reference texts.

METEOR:
Scores range from approximately 0.062 to 0.127. These low scores indicate that the generated content aligns weakly with the reference, reflecting potential limitations in coherence or content relevancy.

Perplexity:
Values ranges from 3.25 to 3.61.This means the model is fairly good at predicting the next word, but there is still room for improvement in making the text more fluent and smooth.
The average perplexity value of 3.4677 indicates that the model has a relatively low level of uncertainty in predicting the next word, suggesting better fluency and coherence in its generated text. 

Entropy:
Values ranges from 4.31 to 4.39,indicating that the model is producing text with moderate diversity. The average entropy of 4.3356 suggests that the text has a balanced amount of variation, without being overly random or repetitive.


### Diversity and Variety

In the context of poem generation, diversity and variety refer to the richness, uniqueness, and creativity of the generated text, which are critical for producing compelling, engaging, and original poetry.

* Self-BLEU : Evaluates how much overlap exists between different samples generated by the same model.
* Distinct n-grams: Measures the percentage of distinct n-grams in the generated text compared to the total n-grams. Higher distinctness means more diversity.

In [18]:
# Self-BLEU Score
def self_bleu(generated_texts, n=1):
    all_ngrams = []
    for text in generated_texts:
        if not isinstance(text, str):
            text = str(text)  # Ensure text is a string
        tokens = text.split()  # Tokenize text
        ngrams_list = list(ngrams(tokens, n))  # Create n-grams
        all_ngrams.append(collections.Counter(ngrams_list))
    
    # Compute Self-BLEU score
    score = 0
    for i, counter in enumerate(all_ngrams):
        others = all_ngrams[:i] + all_ngrams[i+1:]  # All other generated poems except the current one
        union_ngrams = collections.Counter()
        for other in others:
            union_ngrams.update(other)
        
        # Avoid division by zero
        if sum(union_ngrams.values()) > 0:
            overlap = sum(min(count, union_ngrams[gram]) for gram, count in counter.items())
            score += overlap / sum(union_ngrams.values())
    
    return score / len(generated_texts) if len(generated_texts) > 1 else 0

# Distinct-N Score
def distinct_n(generated_texts, n=2):
    ngrams_set = set()
    total_ngrams = 0
    
    for text in generated_texts:
        if not isinstance(text, str):
            text = str(text)  # Ensure text is a string
        tokens = text.split()  # Tokenize text
        ngrams_list = list(ngrams(tokens, n))  # Create n-grams
        ngrams_set.update(ngrams_list)  # Add n-grams to the set
        total_ngrams += len(ngrams_list)
    
    return len(ngrams_set) / total_ngrams if total_ngrams > 0 else 0

In [22]:
# Function to evaluate Self-BLEU and Distinct-N for a model
def evaluate_model_metrics(data, model_name_to_evaluate):
    results = []

    # Collect all poems generated by the same model across different prompts
    generated_poems = data[model_name_to_evaluate].tolist()

    # Calculate Self-BLEU and Distinct-N for the poems generated by the model
    self_bleu_score = self_bleu(generated_poems, n=1)  # For unigrams (Self-BLEU n=1)
    distinct_2_score = distinct_n(generated_poems, n=2)  # For distinct-2 (bigrams)
    
    for index, row in data.iterrows():
        prompt = row["Prompt/Model"]
        
        results.append({
            "Model": model_name_to_evaluate,
            "Prompt": prompt,
            "Self-BLEU (n=1)": round(self_bleu_score, 5),
            "Distinct-2": round(distinct_2_score, 5)
        })

    results_df = pd.DataFrame(results)
    
    evaluation_title = f"Evaluation Results for Model: {model_name_to_evaluate}"
    
    print(evaluation_title)
    print(results_df)

model_name_to_evaluate = "Llama 3.2 3B Instruct Q8" 
evaluate_model_metrics(data, model_name_to_evaluate)

Evaluation Results for Model: Llama 3.2 3B Instruct Q8
                      Model  \
0  Llama 3.2 3B Instruct Q8   
1  Llama 3.2 3B Instruct Q8   
2  Llama 3.2 3B Instruct Q8   

                                              Prompt  Self-BLEU (n=1)  \
0  Write a Shakespearean sonnet \nabout courage s...          0.19815   
1  Write a Shakespearean sonnet \nabout wonder se...          0.19815   
2  Write a Shakespearean sonnet \nabout loss set ...          0.19815   

   Distinct-2  
0     0.90805  
1     0.90805  
2     0.90805  


The model demonstrates strong lexical diversity (as seen with the high Distinct-2 score), suggesting it can generate varied outputs.
The relatively low Self-BLEU score implies the model avoids excessive repetition across different prompts, further supporting its ability to generate diverse and contextually distinct text.

1.  Self-BLEU : This metric measures the overlap of unigrams (single words) within the text generated by the model for different prompts. A higher Self-BLEU score indicates that the model's outputs are more repetitive or similar across different prompts. In this case, the Self-BLEU value of 0.19815 suggests relatively low similarity, indicating some diversity in the generated outputs.
2.  Distinct-2: This metric calculates the ratio of unique bigrams (two-word combinations) to the total number of bigrams in the generated text. It measures lexical diversity, where a higher value signifies more diverse and less repetitive text. Here, the value of 0.90805 indicates a high degree of lexical diversity, meaning the model produces text that is varied and not overly repetitive.

### Poetic Structure:

The script evaluates the rhyming scheme of the generated poems.

* Rhyming Scheme - The rhyming scheme refers to the pattern of rhymes at the end of each line in the poem.
* Syllable Count - This step focuses on counting the syllables in each line of the poem. In structured poetry like sonnets, the number of syllables per line is important.

### Cleaning the sonnet and storing as a list

In [20]:
import pandas as pd
import string

# Function to clean and split poems
def process_poem(poem):
    if isinstance(poem, str):  # Ensure the input is a string
        lines = [
            line.translate(str.maketrans('', '', string.punctuation))
            for line in poem.strip().split('\n') if line.strip()
        ]
        return poem, lines
    else:
        return None, []  # Handle NaN or non-string values


# Filter the data for the selected model
selected_poems = data[model_name_to_evaluate].head(3)  # Select the first 3 poems for the model

# Process the selected poems and assign lines to variables
poem1, lines1 = process_poem(selected_poems.iloc[0])
poem2, lines2 = process_poem(selected_poems.iloc[1])
poem3, lines3 = process_poem(selected_poems.iloc[2])

# Print the processed lines for poem1
print("Poem 1:")
print(f"  Original Poem: {poem1}")
print(f"  Processed Lines: {lines1}")


Poem 1:
  Original Poem: Upon the field of strife, where metal meets,
And fierce contention holds its bitter breath,
There courage stands, a lion's heart it beats,
In soldiers' bosoms, 'gainst all odds they wrench.

With every step that falls upon the earth,
A resounding echo of resolve doth rise,
As brave hearts march to meet their mortal worth,
And face the specter of eternity with eyes.

Through smoke and fire, through noise and clashing steel,
Their unwavering gaze doth pierce the fray,
For they fight not for themselves, but for the real,
To uphold the virtues that shall never sway.

  Processed Lines: ['Upon the field of strife where metal meets', 'And fierce contention holds its bitter breath', 'There courage stands a lions heart it beats', 'In soldiers bosoms gainst all odds they wrench', 'With every step that falls upon the earth', 'A resounding echo of resolve doth rise', 'As brave hearts march to meet their mortal worth', 'And face the specter of eternity with eyes', 'Through

### Poetic Structure:

A Shakespearean Sonnet is a 14 line poem which follows a consitent rhyme scheme pattern of ABAB CDCD EFEF GG and each line is 10 syllables long written in iambic pentameter.

#### Finding the Rhyming Scheme

In [21]:
import pronouncing

poem_list = [lines1, lines2, lines3]

def get_rhyming_scheme(lines):
    # Extract the last word of each line
    last_words = [line.split()[-1] for line in lines]
    rhyme_dict = {}
    scheme = []
    current_letter = 'A'

    for word in last_words:
        rhymes = pronouncing.rhymes(word)
        # Check if the word rhymes with any previously seen words
        for key, letter in rhyme_dict.items():
            if word in rhymes or key in rhymes:
                scheme.append(letter)
                break
        else:
            # Assign a new letter if no rhyming word is found
            rhyme_dict[word] = current_letter
            scheme.append(current_letter)
            current_letter = chr(ord(current_letter) + 1)

    rhyme_scheme = ''.join(scheme)
    return '\n'.join([rhyme_scheme[i:i+4] for i in range(0, len(rhyme_scheme), 4)])


# Process each poem and print rhyming schemes
for i, poem in enumerate(poem_list, 1):
    rhyme_scheme = get_rhyming_scheme(poem)
    print(f"Rhyming Scheme for Poem {i}: \n{rhyme_scheme}")


Rhyming Scheme for Poem 1: 
ABAC
DEDE
FGFG
Rhyming Scheme for Poem 2: 
ABAB
BCBC
BDBE
FF
Rhyming Scheme for Poem 3: 
ABCD
EFGH
IJKL
MNOP
QRSI


When we compare the rhyming schemes of these poems to a Shakespearean sonnet, we can see clear differences:

**Poem 1 (ABAC DEDE FGFG):**  
Differs significantly in structure: lacks the Shakespearean alternating rhyme in its stanzas and the concluding couplet. The rhyme scheme alternates inconsistently and does not group into the clear stanzas and couplet structure of a sonnet.

**Poem 2 (ABAB BCBC BDBE FF):**  
Starts similarly to a Shakespearean sonnet with an alternating ABAB pattern. However, it diverges in the middle with BCBC and BDBE, creating an unusual interlocking structure. Ends with a rhyming couplet (FF), which aligns with the Shakespearean form, but the preceding stanzas are not standard.

**Poem 3 (ABCD EFGH IJKL MNOP QRSI):**  
Completely diverges from a Shakespearean sonnet's structure. Exhibits no repetition of rhymes or stanzas, with each section introducing entirely new rhymes. The rhyme scheme is non-traditional and lacks the coherence and symmetry of a Shakespearean sonnet.

#### Finding the Syllable Counts

In [ ]:
import nltk
from nltk.corpus import cmudict
from collections import defaultdict
import re

nltk.download("cmudict")
cmu_dict = cmudict.dict()

def count_syllables(word):
    """
    Count syllables for a given word using the CMU Pronouncing Dictionary.
    """
    word = word.lower()
    if word in cmu_dict:
        return min([len([y for y in x if y[-1].isdigit()]) for x in cmu_dict[word]])
    return 1  

def analyze_syllable_counts(poem):
    """
    Analyze the syllable count for each line in the poem.
    """
    syllable_counts = [sum(count_syllables(word) for word in re.findall(r'\w+', line)) for line in poem]
    return syllable_counts

all_syllable_counts = [analyze_syllable_counts(poem) for poem in poem_list]

for idx, syllable_count in enumerate(all_syllable_counts):
    print(f"Poem {idx + 1} syllable counts: {syllable_count}")

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\pooji\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


Poem 1 syllable counts: [9, 12, 9, 11, 9, 10, 10, 10, 12, 11, 10, 10, 10, 11]
Poem 2 syllable counts: [9, 12, 12, 12, 9, 12, 10, 12, 10, 11, 13, 10, 10, 11]
Poem 3 syllable counts: [10, 9, 10, 10, 10, 10, 11, 10, 12, 10, 10, 11, 11, 12]


Poem 1 :
Deviation from Shakespearean sonnet: Poem 1 has an inconsistent syllable count per line, with some lines having fewer or more than the expected 10 syllables. The lines vary between 9 and 12 syllables, creating an irregular rhythm rather than the steady 10-syllable pattern of iambic pentameter.
The variations in syllable count also disrupt the traditional flow of a Shakespearean sonnet, which depends on a regular meter to create its musicality.

Poem 2 :
Deviation from Shakespearean sonnet: Like Poem 1, Poem 2 exhibits variability in syllable counts, with lines ranging from 9 to 13 syllables. Several lines exceed the expected 10 syllables, particularly in the middle of the poem, where there are multiple 12-syllable lines.
The lack of consistent 10-syllable lines means the poem doesn't maintain the regular rhythm of iambic pentameter and deviates significantly from the traditional Shakespearean structure.

Poem 3 :
Deviation from Shakespearean sonnet: Poem 3 starts with a few lines that closely resemble the iambic pentameter structure with 10 syllables per line. However, there are still fluctuations in syllable counts, with lines containing 9, 11 and 12 syllables. This inconsistency affects the rhythm, which would traditionally follow the iambic pentameter pattern.
Like the other two poems, this breaks from the expected 10-syllable-per-line structure of a Shakespearean sonnet.